In [1]:
import os
import cv2
import json
import random
random.seed(77)

In [6]:
json_dir_name = os.path.expanduser("~/Desktop/joo/data/korad/data/origin/labeling/")
tiff_dir_name = os.path.expanduser("~/Desktop/joo/data/korad/data/origin/images/")
save_path = os.path.expanduser("~/Desktop/joo/data/korad/data/custom/labeling/")

save_dir = "./korad_boxes"

numbers = os.listdir(tiff_dir_name)
numbers.sort()

all_dict = {}

In [3]:
classes = ['AlligatorCrack', 'ConcreteCrack', 'Spalling', 'Efflorescene', 'Exposure']

colors = []
for i in range(len(classes)):
    rd1 = random.randint(0, 256)
    rd2 = random.randint(0, 256)
    rd3 = random.randint(0, 256)
    rd_color = (rd1, rd2, rd3)
    colors.append(rd_color)


labeling 이랑 image랑 file 명 맞춰서 dictionary 담기

In [10]:
for number in numbers:
    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort()
    tiff_list.sort()

    pair_dict = {}
    for index, tiff_file in enumerate(tiff_list):
        index += 1
        for json_file in json_list:
            if json_file.find(tiff_file[:-5]) == -1 : continue
            pair_dict[tiff_file] = [json_file,"C"+number+"_"+str(index).zfill(5)]
            # pair_dict["C"+number+"_"+str(index).zfill(5)] = [number,tiff_file,json_file,]
            
    all_dict[number] = pair_dict


1번 폴더를 제외한 나머지 폴더 3900개 맞아야 함

In [12]:
# 3900 check!!
for i in all_dict:
    print(f"folder {i} count : {len(all_dict[i])}")

folder 01 count : 0
folder 02 count : 3900
folder 03 count : 3900
folder 04 count : 3900
folder 05 count : 3900
folder 06 count : 3900


In [187]:
# json 으로 만들기.
# with open(os.path.expanduser(save_dir+"/pair.json"), 'w', encoding='utf-8') as file:
#     json.dump(all_dict, file)
#     pass

### json 불러와서 새로운 json 만들기

In [15]:
error_list = {}

for index, number in enumerate(numbers):
    
    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort() 
    tiff_list.sort()
    
    if len(tiff_list) == 0: continue
    
    for tiff_file in tiff_list:
        json_file,id = all_dict[number].get(tiff_file)
        
        # json 불러와서 새로운 json format 만들기
        with open(os.path.expanduser(json_root+"/"+json_file), 'r', encoding='utf-8') as f:
            img_shape = cv2.imread(tiff_root+"/"+tiff_file)
            height, width,_ = img_shape.shape
    
            data = json.load(f)
            
            json_data = {}
            json_data['images'] = {'id':id,"width":width,"height":height,"file_name":id+".tiff"}
            json_data['labels'] = []
            try:
                for box_info in data['labels']:
                    label = box_info['label']
                    
                    box_info = box_info['boundingPoly']['vertices']
                    
                    xmin = box_info[0]['x']
                    xmax = box_info[2]['x']
                    ymin = box_info[0]['y']
                    ymax = box_info[2]['y']
                    
                    coordinate = [{"xmin":xmin,"ymin":ymin,"xmax":xmax,"ymax":ymax}]
                    json_data['labels'].append({"label": label, "boundingPoly":{"type":"RECT", "vertices":coordinate}})
            except KeyError:
                error_list[id] = [number, tiff_file, json_file]

        # 새로운 json format 저장하기
        # print(save_path+number+"/"+id+".json")
        with open(os.path.expanduser(save_path+number+"/"+id+".json"), 'w', encoding='utf-8') as f:
            json.dump(json_data,f)

### 에러 파일들 확인

In [ ]:
error_list
with open(os.path.expanduser("./korad_boxes/error_list.json"), 'w', encoding='utf-8') as f:
    json.dump(error_list,f)

### tiff 이름 변경

In [208]:
for number in numbers:
    
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    tiff_list.sort()
    
    if len(tiff_list) == 0: continue
    
    for tiff_file in tiff_list:
        
        json_file,id = all_dict[number][tiff_file]
        # print(tiff_root)
        # print(json_file)
        # print(id)

        # os.rename(tiff_root+"/"+tiff_file,tiff_root+"/"+id+tiff_file[-5:])